In [1]:
#!pip install matplotlib-venn
import numpy as np
import pandas as pd
import zipfile
import json

In [2]:
comp = {'Monthly': 12, 'Yearly': 1, 'Weekly':52}

def comp_level(x):
    if x[salaryType] != x[salaryType]:
        return x[salaryType]   
    return x[salary] * comp[x[salaryType]] / x[0]

In [3]:
year = 2021
category= 'Programming'
colworked = 'LanguageHaveWorkedWith'
colwanted = 'LanguageWantToWorkWith'
currency = 'Currency'
salary = "CompTotal"
salaryType = "CompFreq"

In [4]:
# Parameters
year = 2018
colworked = "DatabaseWorkedWith"
colwanted = "DatabaseDesireNextYear"
currency = "CurrencySymbol"
salary = "Salary"
salaryType = "SalaryType"
category = "Database"


Cargamos los datos y mostramos su contenido

In [5]:

zf = zipfile.ZipFile(f'../surveys/stack-overflow-developer-survey-{year}.zip') 
devs = pd.read_csv(zf.open('survey_results_public.csv'))
# Cargamos los ratios de conversion de las monedas a 1 de Diciembre de cada año.
with open(f'../rates/rates-{year}.json', 'r') as f:
  data = json.loads(f.read())
rates = pd.json_normalize(data['rates']).transpose()

/home/gpt/app/anaconda3/envs/uoc20202pec1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Eliminamos los nulos de las dos  propiedades que nos intersan

In [6]:
devs[colworked] = devs[colworked].fillna("")
devs[colwanted] = devs[colwanted].fillna("")
devs[salary]= devs[salary].replace("nan", np.NaN)
devs[salary]= pd.to_numeric(devs[salary], errors='coerce')

In [7]:
lang_worked = devs[colworked].str.replace(' ', '').str.get_dummies(sep=';')
lang_want = devs[colwanted].str.replace(' ', '').str.get_dummies(sep=';')

Se nivela la compensacion para obtener el valor anual aproximado en USD, aplicando el cambio de divisa cargado del año

In [8]:
if currency:
    devs[currency] = devs[currency].str[:3]
    devs = pd.merge(devs, rates, left_on=currency, right_index=True)
    devs['CompTotalUSD'] = devs.apply(comp_level, axis=1)
else:
    devs['CompTotalUSD'] = devs[salary]


Obtenemos el dataframe con los distintos lenguajes y su compensacion

In [9]:
comps = lang_worked.multiply( devs['CompTotalUSD']/1000, axis="index").round(decimals=0).replace(0, np.NaN).median().astype('int')

Calculamos el porcentaje de uso de cada lenguaje respecto del total.
Como vamos a comparar años distintos con encuestados distintos por año, el porcentaje anual es una forma de nivelar y poder realizar la comparacion.

In [10]:
worked = lang_worked.sum() / len(lang_worked)
wanted = lang_want.sum() / len(lang_want)

Unimos los dataframes de lenguajes trabajados en el año, lenguajes en los que se quiere trabajar, la compensacion y el año.

In [11]:
w_df = pd.DataFrame(worked, columns= ['Worked'])
w_df['Worked_per'] = (w_df['Worked'] * 100).round(1)
wa_df = pd.DataFrame(wanted, columns= ['Wanted'])
wa_df['Wanted_per'] = (w_df['Worked'] * 100).round(1)
langs = pd.concat([w_df, wa_df, comps], axis=1)
langs['Year'] = year
langs.rename(columns={0: "Salary"}, inplace = True)
langs['Category'] = category

Exportamos los datos procesados del año.

In [12]:
langs.to_csv(f'../out/{year}_{category}.csv', index=True, index_label='Language')